In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
SPARK_HOME = '/home/jose/Frameworks/spark-3.0.2-bin-hadoop2.7'

In [ ]:
# import os
# os.environ['PYSPARK_SUBMIT_ARGS'] = 'pyspark-shell'

import findspark
findspark.init(SPARK_HOME)

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

config = SparkConf() \
                    .setMaster('local[*]') \
                    .setAppName('Spark Streaming') \
                    .setAll([('spark.executor.memory', '2G'),
                            ('spark.driver.memory', '2G'),
                            ('spark.driver.maxResultSize', '1G')])

sc = SparkContext(conf=config)
spark = SparkSession(sc)

spark

## Structured Streaming

## Kafka

In [ ]:
import time 
from pyspark.streaming import StreamingContext

ssc = StreamingContext(sc, 1)

# Create the queue through which RDDs can be pushed to
# a QueueInputDStream
rddQueue = []
for i in range(5):
    rddQueue += [ssc.sparkContext.parallelize([j for j in range(1, 1001)], 10)]

# Create the QueueInputDStream and use it do some processing
inputStream = ssc.queueStream(rddQueue)
mappedStream = inputStream.map(lambda x: (x % 10, 1))
reducedStream = mappedStream.reduceByKey(lambda a, b: a + b)
reducedStream.pprint()

ssc.start()
time.sleep(6)
ssc.stop(stopSparkContext=True, stopGraceFully=True)

In [ ]:
spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "<server:ip") \
    .option("subscribe", "topic1") \
    .option("startingOffsets", "latest") \
    .option("minPartitions", "10") \
    .option("failOnDataLoss", "true") \
    .load()